In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
batch_size = 64
num_epochs = 4  
learning_rate = 0.001
data_dir = 'tomato'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [4]:
train_dataset = datasets.ImageFolder(root='tomato/train', transform=transform)
val_dataset = datasets.ImageFolder(root='tomato/val', transform=transform)


In [5]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

In [6]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(128 * 16 * 16, 512)  
        self.fc2 = nn.Linear(512, len(train_dataset.classes)) 

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 16 * 16) 
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=4):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_predictions = 0
        total_samples = 0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            _, predicted = torch.max(outputs, 1)
            
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()
        
        average_loss = running_loss / len(train_loader)
        accuracy = (correct_predictions / total_samples) * 100
        
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {average_loss:.4f}, Accuracy: {accuracy:.2f}%')
        
train_model(model, train_loader, criterion, optimizer, num_epochs=10)

Epoch 1/10, Loss: 0.9166, Accuracy: 68.43%
Epoch 2/10, Loss: 0.3626, Accuracy: 87.42%
Epoch 3/10, Loss: 0.2364, Accuracy: 92.00%
Epoch 4/10, Loss: 0.1511, Accuracy: 94.76%
Epoch 5/10, Loss: 0.1104, Accuracy: 96.09%
Epoch 6/10, Loss: 0.0878, Accuracy: 97.15%
Epoch 7/10, Loss: 0.0663, Accuracy: 97.67%
Epoch 8/10, Loss: 0.0638, Accuracy: 97.85%
Epoch 9/10, Loss: 0.0335, Accuracy: 98.86%
Epoch 10/10, Loss: 0.0531, Accuracy: 98.21%


In [9]:
def evaluate_model(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy: {100 * correct / total:.2f}%')

evaluate_model(model, val_loader)

Accuracy: 91.00%


In [10]:
torch.save(model, './models/torch_cnn_v1.pth')